# Submission - Inquisitive_Turtles

In [64]:
import pandas as pd

In [65]:
train_df = pd.read_csv('/kaggle/input/data-insurance/train.csv')
test_df = pd.read_csv('/kaggle/input/test-data/test (1).csv')

In [66]:
print('\ntrain df stats')
print(train_df.describe())
print('\ntest df stats')
print(test_df.describe())


train df stats
                 id           Age  Annual Income  Number of Dependents  \
count  1.200000e+06  1.181295e+06   1.155051e+06          1.090328e+06   
mean   5.999995e+05  4.114556e+01   3.274522e+04          2.009934e+00   
std    3.464103e+05  1.353995e+01   3.217951e+04          1.417338e+00   
min    0.000000e+00  1.800000e+01   1.000000e+00          0.000000e+00   
25%    2.999998e+05  3.000000e+01   8.001000e+03          1.000000e+00   
50%    5.999995e+05  4.100000e+01   2.391100e+04          2.000000e+00   
75%    8.999992e+05  5.300000e+01   4.463400e+04          3.000000e+00   
max    1.199999e+06  6.400000e+01   1.499970e+05          4.000000e+00   

       Health Score  Previous Claims   Vehicle Age  Credit Score  \
count  1.125924e+06    835971.000000  1.199994e+06  1.062118e+06   
mean   2.561391e+01         1.002689  9.569889e+00  5.929244e+02   
std    1.220346e+01         0.982840  5.776189e+00  1.499819e+02   
min    2.012237e+00         0.000000  0.00000

In [67]:
def check_missing(df):
    missing_values = df.isnull().sum()
    cols_with_missing = missing_values[missing_values > 0]
    print(cols_with_missing)
    print('\n')
print('\nchecking missing values for train df')
check_missing(train_df)
print('\nchecking missing values for test df')
check_missing(test_df)


checking missing values for train df
Age                      18705
Annual Income            44949
Marital Status           18529
Number of Dependents    109672
Occupation              358075
Health Score             74076
Previous Claims         364029
Vehicle Age                  6
Credit Score            137882
Insurance Duration           1
Customer Feedback        77824
dtype: int64



checking missing values for test df
Age                      12489
Annual Income            29860
Marital Status           12336
Number of Dependents     73130
Occupation              239125
Health Score             49449
Previous Claims         242802
Vehicle Age                  3
Credit Score             91451
Insurance Duration           2
Customer Feedback        52276
dtype: int64




In [68]:
train_df = train_df.dropna(subset = 'Premium Amount')

In [69]:
def check_outliers(df):
    Q1 = df['Previous Claims'].quantile(0.25)
    Q3 = df['Previous Claims'].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    df_cleaned = df[(train_df['Previous Claims'] >= lower_bound) & (df['Previous Claims'] <= upper_bound)]
    df_outliers = df[(train_df['Previous Claims'] < lower_bound) | (df['Previous Claims'] > upper_bound)]

    return df_cleaned,df_outliers

train_df_cleaned, train_df_outliers = check_outliers(train_df)
print('number of outliers in train df: ',train_df_outliers['Previous Claims'].count())
print("Original DataFrame with outliers in train df:",train_df['Previous Claims'].count())
print("DataFrame after removing outliers in train df:",train_df_cleaned['Previous Claims'].count())

test_df_cleaned, test_df_outliers = check_outliers(test_df)
print('number of outliers in test df: ',test_df_outliers['Previous Claims'].count())
print("Original DataFrame with outliers in test df:",test_df['Previous Claims'].count())
print("DataFrame after removing outliers in test df:",test_df_cleaned['Previous Claims'].count())

number of outliers in train df:  369
Original DataFrame with outliers in train df: 835971
DataFrame after removing outliers in train df: 835602


<ipython-input-69-136524a8c1e1>:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_cleaned = df[(train_df['Previous Claims'] >= lower_bound) & (df['Previous Claims'] <= upper_bound)]


number of outliers in test df:  244
Original DataFrame with outliers in test df: 557198
DataFrame after removing outliers in test df: 388260


<ipython-input-69-136524a8c1e1>:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_outliers = df[(train_df['Previous Claims'] < lower_bound) | (df['Previous Claims'] > upper_bound)]


In [70]:
print('\n cleaned train df stats')
print(train_df_cleaned.describe())
print('\n cleaned test df stats')
print(test_df_cleaned.describe())


 cleaned train df stats
                 id            Age  Annual Income  Number of Dependents  \
count  8.356020e+05  822647.000000  803658.000000         758510.000000   
mean   5.999803e+05      41.133822   33043.037242              2.013376   
std    3.463694e+05      13.537680   32655.711592              1.417262   
min    0.000000e+00      18.000000       1.000000              0.000000   
25%    2.999072e+05      30.000000    7765.000000              1.000000   
50%    5.999345e+05      41.000000   23962.000000              2.000000   
75%    8.997585e+05      53.000000   44934.000000              3.000000   
max    1.199999e+06      64.000000  149997.000000              4.000000   

        Health Score  Previous Claims    Vehicle Age   Credit Score  \
count  784628.000000    835602.000000  835597.000000  738880.000000   
mean       25.632917         1.000390       9.559620     594.081544   
std        12.225718         0.976897       5.776622     149.648098   
min         2.0

In [71]:
mean_imputation_cols = ['Age','Number of Dependents','Credit Score','Credit Score']

In [72]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')
train_df_cleaned[mean_imputation_cols]= imputer.fit_transform(train_df_cleaned[mean_imputation_cols])
test_df_cleaned[mean_imputation_cols]= imputer.transform(test_df_cleaned[mean_imputation_cols])
print(train_df_cleaned.head(5))
print(test_df_cleaned.head(5))


   id   Age  Gender  Annual Income Marital Status  Number of Dependents  \
0   0  19.0  Female        10049.0        Married                   1.0   
1   1  39.0  Female        31678.0       Divorced                   3.0   
2   2  23.0    Male        25602.0       Divorced                   3.0   
3   3  21.0    Male       141855.0        Married                   2.0   
4   4  21.0    Male        39651.0         Single                   1.0   

  Education Level     Occupation  Health Score  Location  ... Previous Claims  \
0      Bachelor's  Self-Employed     22.598761     Urban  ...             2.0   
1        Master's            NaN     15.569731     Rural  ...             1.0   
2     High School  Self-Employed     47.177549  Suburban  ...             1.0   
3      Bachelor's            NaN     10.938144     Rural  ...             1.0   
4      Bachelor's  Self-Employed     20.376094     Rural  ...             0.0   

   Vehicle Age  Credit Score  Insurance Duration           Pol

<ipython-input-72-28baafd5655f>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_cleaned[mean_imputation_cols]= imputer.fit_transform(train_df_cleaned[mean_imputation_cols])
<ipython-input-72-28baafd5655f>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_cleaned[mean_imputation_cols]= imputer.transform(test_df_cleaned[mean_imputation_cols])


In [73]:
print('\nchecking missing values for cleaned train df')
check_missing(train_df_cleaned)
print('\nchecking missing values for cleaned test df')
check_missing(test_df_cleaned)


checking missing values for cleaned train df
Annual Income          31944
Marital Status         13213
Occupation            249116
Health Score           50974
Vehicle Age                5
Insurance Duration         1
Customer Feedback      54935
dtype: int64



checking missing values for cleaned test df
Annual Income          14840
Marital Status          6136
Occupation            116236
Health Score           23904
Vehicle Age                2
Insurance Duration         1
Customer Feedback      25882
dtype: int64




In [74]:
mode_imputation_cols=['Annual Income','Health Score','Marital Status','Occupation', 'Customer Feedback']

In [75]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='most_frequent')
train_df_cleaned[mode_imputation_cols]= imputer.fit_transform(train_df_cleaned[mode_imputation_cols])
test_df_cleaned[mode_imputation_cols]= imputer.transform(test_df_cleaned[mode_imputation_cols])
print(train_df_cleaned.head(5))
print(test_df_cleaned.head(5))


<ipython-input-75-df35e6945d4f>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_cleaned[mode_imputation_cols]= imputer.fit_transform(train_df_cleaned[mode_imputation_cols])


   id   Age  Gender Annual Income Marital Status  Number of Dependents  \
0   0  19.0  Female       10049.0        Married                   1.0   
1   1  39.0  Female       31678.0       Divorced                   3.0   
2   2  23.0    Male       25602.0       Divorced                   3.0   
3   3  21.0    Male      141855.0        Married                   2.0   
4   4  21.0    Male       39651.0         Single                   1.0   

  Education Level     Occupation Health Score  Location  ... Previous Claims  \
0      Bachelor's  Self-Employed    22.598761     Urban  ...             2.0   
1        Master's  Self-Employed    15.569731     Rural  ...             1.0   
2     High School  Self-Employed    47.177549  Suburban  ...             1.0   
3      Bachelor's  Self-Employed    10.938144     Rural  ...             1.0   
4      Bachelor's  Self-Employed    20.376094     Rural  ...             0.0   

   Vehicle Age  Credit Score  Insurance Duration           Policy Start Da

<ipython-input-75-df35e6945d4f>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_cleaned[mode_imputation_cols]= imputer.transform(test_df_cleaned[mode_imputation_cols])


In [76]:
print('\nchecking missing values for cleaned train df')
check_missing(train_df_cleaned)
print('\nchecking missing values for cleaned test df')
check_missing(test_df_cleaned)


checking missing values for cleaned train df
Vehicle Age           5
Insurance Duration    1
dtype: int64



checking missing values for cleaned test df
Vehicle Age           2
Insurance Duration    1
dtype: int64




In [77]:
train_df_cleaned = train_df_cleaned.dropna(subset=['Vehicle Age','Insurance Duration'])
test_df_cleaned = test_df_cleaned.dropna(subset=['Vehicle Age','Insurance Duration'])


In [78]:
print('\nchecking missing values for cleaned train df')
check_missing(train_df_cleaned)
print('\nchecking missing values for cleaned test df')
check_missing(test_df_cleaned)


checking missing values for cleaned train df
Series([], dtype: int64)



checking missing values for cleaned test df
Series([], dtype: int64)




In [79]:
train_df_cleaned.describe()

,id,Age,Number of Dependents,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Premium Amount
count,8.355960e+05,835596.000000,835596.000000,835596.000000,835596.000000,835596.000000,835596.000000,835596.000000
mean,5.999812e+05,41.133787,2.013375,1.000388,9.559610,594.081922,5.014704,1113.586324
std,3.463690e+05,13.432306,1.350305,0.976896,5.776618,140.720824,2.594814,870.258271
min,0.000000e+00,18.000000,0.000000,0.000000,0.000000,300.000000,1.000000,20.000000
25%,2.999078e+05,30.000000,1.000000,0.000000,5.000000,486.000000,3.000000,519.000000
50%,5.999370e+05,41.133822,2.000000,1.000000,10.000000,594.081544,5.000000,882.000000
75%,8.997592e+05,53.000000,3.000000,2.000000,15.000000,706.000000,7.000000,1523.000000
max,1.199999e+06,64.000000,4.000000,5.000000,19.000000,849.000000,9.000000,4999.000000


In [80]:
train_df_cleaned.dtypes

id                        int64
Age                     float64
Gender                   object
Annual Income            object
Marital Status           object
Number of Dependents    float64
Education Level          object
Occupation               object
Health Score             object
Location                 object
Policy Type              object
Previous Claims         float64
Vehicle Age             float64
Credit Score            float64
Insurance Duration      float64
Policy Start Date        object
Customer Feedback        object
Smoking Status           object
Exercise Frequency       object
Property Type            object
Premium Amount          float64
dtype: object

In [81]:
from sklearn.preprocessing import OneHotEncoder
encoding_cols=['Customer Feedback','Occupation','Education Level','Marital Status',
               'Property Type','Exercise Frequency','Smoking Status','Policy Type','Gender','Location']

encoder = OneHotEncoder(sparse=False)

train_encoded_array = encoder.fit_transform(train_df_cleaned[encoding_cols])
test_encoded_array = encoder.transform(test_df_cleaned[encoding_cols])

train_encoded_df = pd.DataFrame(train_encoded_array, columns=encoder.get_feature_names_out(encoding_cols))
test_encoded_df = pd.DataFrame(test_encoded_array, columns=encoder.get_feature_names_out(encoding_cols))

train_df_cleaned = pd.concat([train_df_cleaned.drop(columns=encoding_cols), train_encoded_df], axis=1)
test_df_cleaned = pd.concat([test_df_cleaned.drop(columns=encoding_cols), test_encoded_df], axis=1)

print(train_df_cleaned.columns)
print(test_df_cleaned.columns)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Index(['id', 'Age', 'Annual Income', 'Number of Dependents', 'Health Score',
       'Previous Claims', 'Vehicle Age', 'Credit Score', 'Insurance Duration',
       'Policy Start Date', 'Premium Amount', 'Customer Feedback_Average',
       'Customer Feedback_Good', 'Customer Feedback_Poor',
       'Occupation_Employed', 'Occupation_Self-Employed',
       'Occupation_Unemployed', 'Education Level_Bachelor's',
       'Education Level_High School', 'Education Level_Master's',
       'Education Level_PhD', 'Marital Status_Divorced',
       'Marital Status_Married', 'Marital Status_Single',
       'Property Type_Apartment', 'Property Type_Condo', 'Property Type_House',
       'Exercise Frequency_Daily', 'Exercise Frequency_Monthly',
       'Exercise Frequency_Rarely', 'Exercise Frequency_Weekly',
       'Smoking Status_No', 'Smoking Status_Yes', 'Policy Type_Basic',
       'Policy Type_Comprehensive', 'Policy Type_Premium', 'Gender_Female',
       'Gender_Male', 'Location_Rural', 'Location_Su

In [82]:
train_df_cleaned.head(5)

,id,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,...,Smoking Status_No,Smoking Status_Yes,Policy Type_Basic,Policy Type_Comprehensive,Policy Type_Premium,Gender_Female,Gender_Male,Location_Rural,Location_Suburban,Location_Urban
0,0.0,19.0,10049.0,1.0,22.598761,2.0,17.0,372.000000,5.0,2023-12-23 15:21:39.134960,...,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
1,1.0,39.0,31678.0,3.0,15.569731,1.0,12.0,694.000000,2.0,2023-06-12 15:21:39.111551,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
2,2.0,23.0,25602.0,3.0,47.177549,1.0,14.0,594.081544,3.0,2023-09-30 15:21:39.221386,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,3.0,21.0,141855.0,2.0,10.938144,1.0,0.0,367.000000,1.0,2024-06-12 15:21:39.226954,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,4.0,21.0,39651.0,1.0,20.376094,0.0,8.0,598.000000,4.0,2021-12-01 15:21:39.252145,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0


In [83]:
test_df_cleaned.head(5)

,id,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,...,Smoking Status_No,Smoking Status_Yes,Policy Type_Basic,Policy Type_Comprehensive,Policy Type_Premium,Gender_Female,Gender_Male,Location_Rural,Location_Suburban,Location_Urban
3,1200003.0,28.0,30424.0,3.0,5.136225,1.0,3.0,770.0,5.0,2023-10-25 15:21:39.134960,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
6,1200006.0,21.0,5896.0,3.0,14.773963,1.0,14.0,321.0,8.0,2023-11-12 15:21:39.288099,...,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
8,1200008.0,32.0,76463.0,4.0,22.988787,0.0,1.0,540.0,2.0,2021-07-08 15:21:39.169102,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
9,1200009.0,48.0,27982.0,2.0,43.567433,1.0,15.0,757.0,7.0,2021-02-15 15:21:39.134960,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
10,1200010.0,44.0,24238.0,0.0,20.769332,0.0,11.0,408.0,3.0,2021-06-25 15:21:39.134960,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0


In [84]:
train_df_cleaned.dtypes

id                             float64
Age                            float64
Annual Income                   object
Number of Dependents           float64
Health Score                    object
Previous Claims                float64
Vehicle Age                    float64
Credit Score                   float64
Insurance Duration             float64
Policy Start Date               object
Premium Amount                 float64
Customer Feedback_Average      float64
Customer Feedback_Good         float64
Customer Feedback_Poor         float64
Occupation_Employed            float64
Occupation_Self-Employed       float64
Occupation_Unemployed          float64
Education Level_Bachelor's     float64
Education Level_High School    float64
Education Level_Master's       float64
Education Level_PhD            float64
Marital Status_Divorced        float64
Marital Status_Married         float64
Marital Status_Single          float64
Property Type_Apartment        float64
Property Type_Condo      

In [85]:
train_df_cleaned = train_df_cleaned.dropna()

In [86]:
train_df_cleaned.describe()

,id,Age,Number of Dependents,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Premium Amount,Customer Feedback_Average,Customer Feedback_Good,...,Smoking Status_No,Smoking Status_Yes,Policy Type_Basic,Policy Type_Comprehensive,Policy Type_Premium,Gender_Female,Gender_Male,Location_Rural,Location_Suburban,Location_Urban
count,581884.000000,581884.000000,581884.000000,581884.000000,581884.000000,581884.000000,581884.000000,581884.000000,581884.000000,581884.000000,...,581884.000000,581884.000000,581884.000000,581884.000000,581884.000000,581884.000000,581884.000000,581884.000000,581884.000000,581884.000000
mean,417834.454536,41.141527,2.013831,0.999955,9.561010,594.010609,5.016130,1113.471011,0.380533,0.307640,...,0.498546,0.501454,0.333551,0.332193,0.334256,0.497637,0.502363,0.333426,0.334666,0.331908
std,241212.137558,13.433054,1.350569,0.976736,5.775115,140.753756,2.594489,870.936558,0.485518,0.461517,...,0.499998,0.499998,0.471482,0.471000,0.471730,0.499995,0.499995,0.471438,0.471874,0.470899
min,0.000000,18.000000,0.000000,0.000000,0.000000,300.000000,1.000000,20.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,208933.750000,30.000000,1.000000,0.000000,5.000000,486.000000,3.000000,518.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,417795.500000,41.133822,2.000000,1.000000,10.000000,594.081544,5.000000,882.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,626883.500000,53.000000,3.000000,2.000000,15.000000,706.000000,7.000000,1523.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,835595.000000,64.000000,4.000000,5.000000,19.000000,849.000000,9.000000,4997.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [87]:
test_df_cleaned = test_df_cleaned.dropna()

In [88]:
test_df_cleaned.describe()

,id,Age,Number of Dependents,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Customer Feedback_Average,Customer Feedback_Good,Customer Feedback_Poor,...,Smoking Status_No,Smoking Status_Yes,Policy Type_Basic,Policy Type_Comprehensive,Policy Type_Premium,Gender_Female,Gender_Male,Location_Rural,Location_Suburban,Location_Urban
count,1.886160e+05,188616.000000,188616.000000,188616.000000,188616.000000,188616.000000,188616.000000,188616.000000,188616.000000,188616.000000,...,188616.000000,188616.000000,188616.000000,188616.000000,188616.000000,188616.000000,188616.000000,188616.000000,188616.000000,188616.000000
mean,1.394318e+06,41.117396,2.018927,1.003727,9.554852,593.565332,5.017782,0.381967,0.306315,0.311718,...,0.498425,0.501575,0.333922,0.331600,0.334479,0.498028,0.501972,0.334107,0.334722,0.331170
std,1.120151e+05,13.433107,1.346322,0.978101,5.781588,141.287133,2.594841,0.485870,0.460964,0.463197,...,0.499999,0.499999,0.471613,0.470789,0.471809,0.499997,0.499997,0.471679,0.471895,0.470635
min,1.200003e+06,18.000000,0.000000,0.000000,0.000000,300.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.297548e+06,30.000000,1.000000,0.000000,5.000000,484.000000,3.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.394276e+06,41.000000,2.000000,1.000000,10.000000,594.081544,5.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,1.491230e+06,53.000000,3.000000,2.000000,15.000000,706.000000,7.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.588256e+06,64.000000,4.000000,5.000000,19.000000,849.000000,9.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [89]:
train_df_cleaned['Number of Dependents'] = train_df_cleaned['Number of Dependents'].astype('int32')
test_df_cleaned['Number of Dependents'] = test_df_cleaned['Number of Dependents'].astype('int32')

In [90]:
train_df_cleaned['Policy Start Date'] = train_df_cleaned['Policy Start Date'].astype('datetime64[ns]')
test_df_cleaned['Policy Start Date'] = test_df_cleaned['Policy Start Date'].astype('datetime64[ns]')

In [91]:
train_df_cleaned['Annual Income'] = train_df_cleaned['Annual Income'].astype('float64')
test_df_cleaned['Annual Income'] = test_df_cleaned['Annual Income'].astype('float64')

In [92]:
train_df_cleaned['Health Score'] = train_df_cleaned['Health Score'].astype('float64')
test_df_cleaned['Health Score'] = test_df_cleaned['Health Score'].astype('float64')

In [93]:
train_df_cleaned.dtypes

id                                    float64
Age                                   float64
Annual Income                         float64
Number of Dependents                    int32
Health Score                          float64
Previous Claims                       float64
Vehicle Age                           float64
Credit Score                          float64
Insurance Duration                    float64
Policy Start Date              datetime64[ns]
Premium Amount                        float64
Customer Feedback_Average             float64
Customer Feedback_Good                float64
Customer Feedback_Poor                float64
Occupation_Employed                   float64
Occupation_Self-Employed              float64
Occupation_Unemployed                 float64
Education Level_Bachelor's            float64
Education Level_High School           float64
Education Level_Master's              float64
Education Level_PhD                   float64
Marital Status_Divorced           

In [94]:
test_df_cleaned.dtypes

id                                    float64
Age                                   float64
Annual Income                         float64
Number of Dependents                    int32
Health Score                          float64
Previous Claims                       float64
Vehicle Age                           float64
Credit Score                          float64
Insurance Duration                    float64
Policy Start Date              datetime64[ns]
Customer Feedback_Average             float64
Customer Feedback_Good                float64
Customer Feedback_Poor                float64
Occupation_Employed                   float64
Occupation_Self-Employed              float64
Occupation_Unemployed                 float64
Education Level_Bachelor's            float64
Education Level_High School           float64
Education Level_Master's              float64
Education Level_PhD                   float64
Marital Status_Divorced               float64
Marital Status_Married            

In [ ]:
skewed_cols=['Annual Income','Health Score','Premium Amount']

train_df_cleaned['Annual Income'].describe()

In [95]:
features=train_df_cleaned.columns.tolist()

In [96]:
features.remove('Premium Amount')

In [100]:
features.remove('Policy Start Date')

In [97]:
target='Premium Amount'

In [ ]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
import lightgbm as lgb
import catboost as cb
import numpy as np
import pandas as pd

# Define the dataset
X = train_df_cleaned[features]  # All feature columns
y = train_df_cleaned[target]    # Target column

# Define skewed and non-skewed columns
skewed_cols = ['Annual Income','Health Score']  # List of skewed features
non_skewed_cols = [col for col in X.columns if col not in skewed_cols]

# Set up models
models = {
    'Gradient Boosting Regressor': GradientBoostingRegressor(random_state=42),
    'XGBoost': xgb.XGBRegressor(random_state=42, verbosity=0),
    'LightGBM': lgb.LGBMRegressor(random_state=42),
    'CatBoost': cb.CatBoostRegressor(learning_rate=0.1, depth=6, iterations=100, verbose=0, random_state=42),
}

# Set up K-Fold Cross-Validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Placeholder for results
results = {}

# Loop through models
for model_name, model in models.items():
    print(f"Evaluating {model_name}...")
    
    # Placeholder for fold-wise RMSEs
    fold_rmse = []
    
    # Loop through folds
    for train_idx, val_idx in kfold.split(X, y):
        # Split data
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        # Transform skewed features
        pt_features = PowerTransformer(method='yeo-johnson')
        X_train_skewed = pt_features.fit_transform(X_train[skewed_cols])
        X_val_skewed = pt_features.transform(X_val[skewed_cols])

        # Transform the target
        pt_target = PowerTransformer(method='yeo-johnson')
        y_train_transformed = pt_target.fit_transform(y_train.values.reshape(-1, 1)).flatten()
        y_val_transformed = pt_target.transform(y_val.values.reshape(-1, 1)).flatten()

        # Combine transformed skewed features and non-skewed features
        X_train_combined = np.hstack((X_train_skewed, X_train[non_skewed_cols].values))
        X_val_combined = np.hstack((X_val_skewed, X_val[non_skewed_cols].values))

        # Train the model
        model.fit(X_train_combined, y_train_transformed)

        # Predict on validation set
        y_pred_transformed = model.predict(X_val_combined)

        # Inverse-transform predictions to original target scale
        y_pred_original = pt_target.inverse_transform(y_pred_transformed.reshape(-1, 1)).flatten()

        # Compute RMSE in the original target space
        rmse = np.sqrt(mean_squared_error(y_val, y_pred_original))
        fold_rmse.append(rmse)
    
    # Store results for the current model
    results[model_name] = {
        'Fold RMSEs': fold_rmse,
        'Mean RMSE': np.mean(fold_rmse),
        'Std RMSE': np.std(fold_rmse),
    }




In [103]:
# Display Results
for model_name, metrics in results.items():
    print(f"\n{model_name}")
    print(f"  Fold RMSEs: {metrics['Fold RMSEs']}")
    print(f"  Mean RMSE: {metrics['Mean RMSE']:.4f}")
    print(f"  Std RMSE: {metrics['Std RMSE']:.4f}")


AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import boxcox, yeojohnson
from scipy.stats import shapiro
import pandas as pd

# Dictionary to track lambda values for inverse transformations
lambda_dict = {}

# Function to visualize the distribution
def plot_hist(df, col):
    plt.hist(df[col], bins=10, color='blue', edgecolor='black', alpha=0.7)
    plt.title(f'Distribution of {col}')
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.show()

# Function to apply transformations
def apply_transformations(df, skewed_cols, lambda_dict=None, calculate_lambda=True):
    if lambda_dict is None:
        lambda_dict = {}

    for col in skewed_cols:
        if col in df.columns:
            transformations = {}
            # Check initial skewness
            print(f"Initial Skewness of {col}: {df[col].skew()}")

            # Apply square root transformation
            df[f'{col}_sqrt'] = np.sqrt(df[col])
            transformations[f'{col}_sqrt'] = df[f'{col}_sqrt'].skew()

            # Apply cube root transformation
            df[f'{col}_cbrt'] = np.cbrt(df[col])
            transformations[f'{col}_cbrt'] = df[f'{col}_cbrt'].skew()

            # Apply log transformation
            df[f'{col}_log'] = np.log1p(df[col])
            transformations[f'{col}_log'] = df[f'{col}_log'].skew()

            # Apply Box-Cox transformation (requires positive values)
            if (df[col] > 0).all():
                if calculate_lambda:
                    df[f'{col}_boxcox'], boxcox_lambda = boxcox(df[col] + 1e-6)
                    lambda_dict[f'{col}_boxcox'] = boxcox_lambda
                else:
                    df[f'{col}_boxcox'] = boxcox(df[col] + 1e-6, lmbda=lambda_dict[f'{col}_boxcox'])
                transformations[f'{col}_boxcox'] = pd.Series(df[f'{col}_boxcox']).skew()

            # Apply Yeo-Johnson transformation
            if calculate_lambda:
                df[f'{col}_yeojohnson'], yeojohnson_lambda = yeojohnson(df[col])
                lambda_dict[f'{col}_yeojohnson'] = yeojohnson_lambda
            else:
                df[f'{col}_yeojohnson'] = yeojohnson(df[col], lmbda=lambda_dict[f'{col}_yeojohnson'])
            transformations[f'{col}_yeojohnson'] = pd.Series(df[f'{col}_yeojohnson']).skew()

            # Plot transformed distributions
            print(f"After transformations for {col}:")
            for key, value in transformations.items():
                print(f"{key}: Skewness = {value}")
                plot_hist(df, key)

    return df, lambda_dict

# Apply transformations to the train dataset and calculate lambda values
print('Applying transformations to train set:')
train_df_cleaned, lambda_dict = apply_transformations(
    train_df_cleaned, skewed_cols, lambda_dict=lambda_dict, calculate_lambda=True
)

# Apply the same transformations to the test dataset using the lambda values from the train dataset
print('Applying transformations to test set:')
test_df_cleaned, _ = apply_transformations(
    test_df_cleaned, skewed_cols, lambda_dict=lambda_dict, calculate_lambda=False
)

# Display the tracked lambda values
print("Tracked Lambda Values:")
for key, value in lambda_dict.items():
    print(f"{key}: lambda = {value}")


Selecting yeo johnson transformation for all the three skewed columns

In [ ]:
train_df_cleaned = train_df_cleaned.drop(columns=['Annual Income_sqrt', 'Annual Income_cbrt', 'Annual Income_log','Annual Income_boxcox', 'Health Score_sqrt',
                              'Health Score_cbrt', 'Health Score_log', 'Health Score_boxcox', 'Premium Amount_sqrt', 'Premium Amount_cbrt',
                              'Premium Amount_log', 'Premium Amount_boxcox','Annual Income','Health Score', 'Premium Amount'])

In [ ]:
test_df_cleaned = test_df_cleaned.drop(columns=['Annual Income_sqrt', 'Annual Income_cbrt', 'Annual Income_log','Annual Income_boxcox', 'Health Score_sqrt',
                              'Health Score_cbrt', 'Health Score_log', 'Health Score_boxcox','Annual Income','Health Score'])

In [ ]:
print(train_df_cleaned.columns)
print(test_df_cleaned.columns)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def plot_correlation_matrix(df, figsize=(10, 8), save_plot=False, filename="correlation_matrix.png"):
    """
    Plot the correlation matrix of a DataFrame.
    Parameters:
        df (pd.DataFrame): The DataFrame containing the features.
        figsize (tuple): Size of the heatmap figure.
        save_plot (bool): Whether to save the plot as a file.
        filename (str): Name of the file to save the plot if save_plot is True.
    """
    # Compute the correlation matrix
    correlation_matrix = df.corr()

    # Create the heatmap
    plt.figure(figsize=figsize)
    sns.heatmap(
        correlation_matrix,
        annot=True,
        fmt=".2f",
        cmap="coolwarm",
        cbar=True,
        square=True,
        linewidths=0.5,
        annot_kws={"size": 8}
    )
    plt.title("Feature Correlation Matrix", fontsize=16)
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)

    if save_plot:
        plt.savefig(filename, bbox_inches="tight")
        print(f"Correlation matrix saved as {filename}")
    else:
        plt.show()

# Example: Visualize correlation for train and test DataFrames
print("Correlation Matrix for Train DataFrame")
plot_correlation_matrix(train_df_cleaned, figsize=(12, 10))

print("Correlation Matrix for Test DataFrame")
plot_correlation_matrix(test_df_cleaned, figsize=(12, 10), save_plot=True, filename="test_correlation_matrix.png")


In [ ]:
train_df_cleaned.columns

In [ ]:
train_df_cleaned = train_df_cleaned.drop(columns=['Policy Start Date'])
test_df_cleaned = test_df_cleaned.drop(columns=['Policy Start Date'])

In [ ]:
pip install tqdm

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
import xgboost as xgb
import lightgbm as lgb
import catboost as cb
from tqdm import tqdm
def apply_advanced_regression_models(train_df, test_df, target_col, features):
    X_train = train_df[features]
    y_train = train_df[target_col]
    X_test = test_df[features]
    
    models = {
        'Gradient Boosting Regressor': GradientBoostingRegressor(),
        'XGBoost': xgb.XGBRegressor(),
        'LightGBM': lgb.LGBMRegressor(),
        'CatBoost': cb.CatBoostRegressor(learning_rate=0.1, depth=6, iterations=100, verbose=0)
    }

    # Dictionary to store predictions and evaluation metrics
    predictions = {}
    results = {}

    # Apply each model and track progress using tqdm
    for model_name, model in tqdm(models.items(), desc="Training Models", total=len(models)):
        print(f"\nTraining {model_name}...")

        if model_name == 'Support Vector Regressor':
            # For SVR, use the scaled data
            model.fit(X_train_scaled, y_train)
            y_pred = model.predict(X_test_scaled)
        else:
            # For other models, use the original features
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

        # Store predictions for the test set
        predictions[model_name] = y_pred

        # If target column is available in test_df, calculate evaluation metrics
        if target_col in test_df.columns:
            y_test = test_df[target_col]
            mae = mean_absolute_error(y_test, y_pred)
            mse = mean_squared_error(y_test, y_pred)
            rmse = np.sqrt(mse)
            r2 = r2_score(y_test, y_pred)

            # Store the results
            results[model_name] = {
                'MAE': mae,
                'MSE': mse,
                'RMSE': rmse,
                'R2 Score': r2
            }

            # Print the evaluation metrics
            print(f"{model_name} Results:")
            print(f"MAE: {mae:.4f}")
            print(f"MSE: {mse:.4f}")
            print(f"RMSE: {rmse:.4f}")
            print(f"R2 Score: {r2:.4f}")

    # If target column is missing in test_df, only return predictions
    if target_col not in test_df.columns:
        predictions_df = pd.DataFrame(predictions)
        print("\nPredictions for Test Data:")
        print(predictions_df.head())
        return predictions_df
    else:
        # Convert results into a DataFrame for better visualization
        if results:
            results_df = pd.DataFrame(results).T
            return results_df
        else:
            return None


In [ ]:
# Define your features and target column (replace with your actual column names)
target_col = 'Premium Amount_yeojohnson'  # Replace with the actual target column
features = [col for col in train_df_cleaned.columns if col != target_col]  # Exclude target column from features

# Apply advanced regression models
predictions_df = apply_advanced_regression_models(train_df_cleaned, test_df_cleaned, target_col, features)

# Display predictions for test data
if predictions_df is not None:
    print("\nPredictions for Test Data:")
    print(predictions_df)


In [ ]:
predictions_df.head(5)

In [ ]:
train_df_cleaned['Premium Amount_yeojohnson'].describe()

In [ ]:
predictions_df.describe()

In [ ]:
predictions_df.describe()

In [ ]:
import numpy as np

# Yeo-Johnson inverse transformation
def yeojohnson_inverse(y_transformed, lmbda):
    y_original = np.zeros_like(y_transformed)
    
    # For y >= 0
    pos_idx = y_transformed >= 0
    if lmbda == 0:
        y_original[pos_idx] = np.exp(y_transformed[pos_idx]) - 1
    else:
        y_original[pos_idx] = (y_transformed[pos_idx] * lmbda + 1)**(1 / lmbda) - 1
    
    # For y < 0
    neg_idx = ~pos_idx
    if lmbda == 2:
        y_original[neg_idx] = 1 - np.exp(-y_transformed[neg_idx])
    else:
        y_original[neg_idx] = 1 - ((-y_transformed[neg_idx] * (2 - lmbda) + 1)**(1 / (2 - lmbda)))
    
    return y_original

# Inverse-transform the 'premium_amount' column for all models
def inverse_transform_premium_amount(predictions, lambda_value):
    for model in predictions.columns:
        predictions[model] = yeojohnson_inverse(predictions[model].values, lambda_value)
    return predictions

# Example usage
# Retrieve the lambda value for the 'premium_amount' column
lambda_value = lambda_dict["Premium Amount_yeojohnson"]  # Ensure this is correct and set during preprocessing

# Inverse-transform the predictions
predictions = inverse_transform_premium_amount(predictions_df, lambda_value)

# Display the inverse-transformed predictions
print(predictions.head())


In [ ]:
train_df['Premium Amount'].describe()

In [ ]:
predictions.head(5)



In [ ]:
lambda_dict

In [ ]:
features=train_df_cleaned.columns.tolist()

In [ ]:
features